In [ ]:
# train_augmented.py
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

# Path to the cleaned CSV that includes original and augmented image paths
CSV_PATH = 'cleaned_augmented_dataset.csv'
df = pd.read_csv(CSV_PATH)

# Optional: Print head to verify
print("Dataset sample:")
print(df.head())

# Scale Age values to help training (especially if Age spans a wide range)
scaler = MinMaxScaler()
df['Age_scaled'] = scaler.fit_transform(df[['Age']])

# Set up ImageDataGenerator with a validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 20% validation
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 20

# Use "Age_scaled" as target if scaling is applied.
# Assume the "Image" column in your CSV contains paths like:
# "artifact_dataset/images/coin/457172.jpg" or "artifact_dataset/augmented_images/coin/aug_457172.jpg"
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col="Image",
    y_col="Age_scaled",
    directory='',   # '' if Image column already contains the full relative path
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    subset="training",
    class_mode='raw'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    x_col="Image",
    y_col="Age_scaled",
    directory='',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    subset="validation",
    class_mode='raw'
)

# Build the image-only model using ResNet50 as base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # regression output for Age
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='mean_squared_error',
              metrics=['mae'])
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("final_artifact_date_model.keras", monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Fine-tuning: unfreeze the last 20 layers and train further
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='mean_squared_error',
              metrics=['mae'])

history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Save the final model
model.save("final_artifact_date_model.keras")
print("Training complete. Final image-only model saved as final_artifact_date_model.keras")
